In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from pathlib import Path

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 20px; background: black;"><i><b style="color:#DEB887;">NFL Bowl 2022 - Plays and Players</b></i></h1></center>

![](https://images.daznservices.com/di/library/sporting_news/e7/b2/2022-nfl-draft-big-board_9ege0v5zsuwa1hofx39butfta.jpg?t=476393145&quality=100)sportingnews.com

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import shap
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import random

In [ ]:
players = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
players.head()

<h1><span class="label label-default" style="background-color:#8B4513;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#DEB887; padding:10px">NFL Abbreviations</span></h1><br>

Abbreviation	Position

QB	Quarterback

RB	Running Back

FB	Fullback

WR	Wide Receiver

TE	Tight End

OL	Offensive Lineman

C	Center

G	Guard

LG	Left Guard

RG	Right Guard

T	Tackle

LT	Left Tackle

RT	Right Tackle

K	Kicker

KR	Kick Returner

DL	Defensive Lineman

DE	Defensive End

DT	Defensive Tackle

NT	Nose Tackle

LB	Linebacker

ILB	Inside Linebacker

OLB	Outside Linebacker

MLB	Middle Linebacker

DB	Defensive Back

CB	Cornerback

FS	Free Safety

SS	Strong Safety

S	Safety

P	Punter

PR	Punt Returner

http://stats.washingtonpost.com/fb/glossary.asp

In [ ]:
mean=np.ceil(players['weight'].mean())
median=np.ceil(players['weight'].median())

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBJ4LtgNbo9atvDby0C-pp5tP0ab0_WnIh4Q&usqp=CAU)sportsnaut.com

In [ ]:
#Code by Chinta https://www.kaggle.com/chinta/mlb-is-player-age-important

plt.figure(figsize=(10, 5))
sns.set_style('white')
hist_plot = sns.histplot(players['weight'], )
hist_plot.axvline(mean, color='r', linestyle='--', linewidth = 4, label = f'mean-{mean}')
hist_plot.axvline(median, color='g', linestyle='-', linewidth = 4, label = f'median-{median}')
plt.suptitle("Players Weight Distribution")
plt.legend();

![](https://nflstatic.s3.amazonaws.com/attachments/ckd0czhw40o680imyjdsqysfy-offensive-vs-defensivev2.max.png)nflflag.com

In [ ]:
scouting = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
scouting.head(2)

<h1><span class="label label-default" style="background-color:#8B4513;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#DEB887; padding:10px">Receptions, Catches, Laterals</span></h1><br>

#That's the moment I'm really going crazy: We have R and C in Ventilator Pressure Prediction Contest. Above we see R and C (kickDirectionIntended, kickDirectionActual,returnDirectionIntended, returnDirectionActual) columns. 

Maybe it's Reception/Catch/Lateral.

"When a player catches (receives) the ball past the line of scrimmage. If a RECEPTION is made behind the line of scrimmage, it is a LATERAL. Another name for a CATCH."

https://en.wikipedia.org/wiki/Glossary_of_American_football_terms

![](https://pbs.twimg.com/media/DMSJFq1WkAAv6BR.jpg)twitter.com

In [ ]:
tracking = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
tracking.head(2)

<h1><span class="label label-default" style="background-color:#8B4513;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#DEB887; padding:10px">Jersey Numbers: Squad Numbers</span></h1><br>

Jersey Numbers:

"Numbers 1 to 19 are worn by quarterbacks, kickers, and punters. Since 2004, wide receivers are also allowed to wear numbers between 10 and 19 if they so choose, even if there is an 80-89 number available. In the rare event that all numbers from 10-19 and 80-89 are taken, receivers are allowed to wear single digit numbers, such as Biren Ealy, current NFL Free Agent, on the Baltimore Ravens then wore a single digit number in the 2009 Preseason."


"Numbers 20 to 49 are worn by running backs, tight ends (who may wear a number between 40-49 when 80-89 are unavailable), cornerbacks and safeties."

"Numbers 50 to 59 are worn by linebackers and centers."

"Numbers 60 to 79 are worn by tackles, guards and defensive linemen (the defensive ends, defensive tackles or nose guard)."

"Numbers 80 to 89 are worn by wide receivers and tight ends."

"Numbers 90 to 99 are worn by linebackers and defensive linemen. In use since 1984."

"Numbers 0 and 00 are no longer used, though they were issued in the NFL before the number standardization in 1973. George Plimpton wore 0 during a brief preseason stint at quarterback for the Detroit Lions. Jim Otto ("aught-oh") wore number "00" during most of his career with the Oakland Raiders. Wide receiver Ken Burrough of the Houston Oilers also wore "00" during his NFL career in the 1970s."

https://americanfootball.fandom.com/wiki/Squad_number

In [ ]:
tracking19 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2019.csv')
tracking19.head(2)

![](http://static1.squarespace.com/static/54aac868e4b00f7c5fbf0a24/54ad9112e4b00b45194e9d3f/558452a5e4b0875de16d1201/1434736328179/?format=1500w)makegamedayeveryday.com

In [ ]:
tracking18 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2018.csv')
tracking18.head(2)

In [ ]:
games = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
games.head()

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRnOP38qQrRtc32f5eG3pzQh0K_iRQOMgq5Wg&usqp=CAU)twitter.com

In [ ]:
plays = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
plays.head(2)

<h1><span class="label label-default" style="background-color:#8B4513;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#DEB887; padding:10px">American Football Terms</span></h1><br>

What is a TOUCHBACK?

"A situation in football in which the ball is down behind the goal line after a kick or intercepted forward pass after which it is put in play by the team defending the goal on its own 20-yard line — compare safety."

Definition of YARD LINE: 

Any of a series of marked or imaginary lines one yard apart on a football field that are parallel to the goal lines and that indicate the distance to the nearest goal line

KICKOFF

A free kick which starts each half, or restarts the game following a touchdown or field goal. The kickoff may be a place kick in American or Canadian football, or a drop kick in American football.

https://www.merriam-webster.com/dictionary

https://en.wikipedia.org/wiki/Glossary_of_American_football_terms

In [ ]:
plays.isnull().sum()

In [ ]:
#word cloud
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in plays.specialTeamsPlayType)
# Create and generate a word cloud image:
wordcloud = WordCloud(max_words=200,colormap='Set3', background_color="black").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
# Display the generated image:
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
playscorr=plays.corr()
playscorr

In [ ]:
plt.figure(figsize=(10,4))
sns.heatmap(plays.corr(),annot=False,cmap='summer')
plt.show()

In [ ]:
#Code by Thor the God of Thunder

cat = []
num = []
for col in tracking.columns:
    if tracking[col].dtype=='O':
        cat.append(col)
    else:
        num.append(col)  
        
        
num 

In [ ]:
#word cloud
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in plays.specialTeamsResult)
# Create and generate a word cloud image:
wordcloud = WordCloud(max_words=200,colormap='GnBu', background_color="white").generate(text)
plt.figure(figsize=(10,6))
plt.figure(figsize=(15,10))
# Display the generated image:
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
#Code by Thor the God of Thunder

#plt.style.use('dark_background') #dark ruined my Shap values 
for col in tracking[num].drop(['playId'],axis=1):
    plt.figure(figsize=(8,5))
    plt.plot(tracking[col].value_counts(),color='Red')
    plt.xlabel(col)
    plt.ylabel('playId')
    plt.tight_layout()
    plt.show()

#In spite of the charts above, I still don't know the meaning of x, y, s, a, dis, o features.

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://www.rookieroad.com/img/football/football-touchback.png',width=400,height=400)

In [ ]:
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

SEED = 99
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

playsmodel = plays.copy()

# read the "object" columns and use labelEncoder to transform to numeric
for col in playsmodel.columns[playsmodel.dtypes == 'object']:
    le = LabelEncoder()
    playsmodel[col] = playsmodel[col].astype(str)
    le.fit(playsmodel[col])
    playsmodel[col] = le.transform(playsmodel[col])

In [ ]:
#change columns names to alphanumeric
playsmodel.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in playsmodel.columns]

In [ ]:
X = playsmodel.drop(['playResult','playId'], axis = 1)
y = playsmodel['playResult']

In [ ]:
lgb_params = {
                    'objective':'binary',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.005,
                    'num_leaves': 20,
                    'max_depth':-1,
                    'subsample':0.9,
                    'n_estimators':2500,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                }

In [ ]:
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

# choose the number of folds, and create a variable to store the auc values and the iteration values.
K = 5
folds = KFold(K, shuffle = True, random_state = SEED)
best_scorecv= 0
best_iteration=0

# Separate data in folds, create train and validation dataframes, train the model and cauculate the mean AUC.
for fold , (train_index,test_index) in enumerate(folds.split(X, y)):
    print('Fold:',fold+1)
          
    X_traincv, X_testcv = X.iloc[train_index], X.iloc[test_index]
    y_traincv, y_testcv = y.iloc[train_index], y.iloc[test_index]
    
    train_data = lgb.Dataset(X_traincv, y_traincv)
    val_data   = lgb.Dataset(X_testcv, y_testcv)
    
    LGBM = lgb.train(lgb_params, train_data, valid_sets=[train_data,val_data], verbose_eval=250)
    best_scorecv += LGBM.best_score['valid_1']['auc']
    best_iteration += LGBM.best_iteration

best_scorecv /= K
best_iteration /= K
print('\n Mean AUC score:', best_scorecv)
print('\n Mean best iteration:', best_iteration)

In [ ]:
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

lgb_params = {
                    'objective':'binary',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.05,
                    'num_leaves': 20,
                    'max_depth':-1,
                    'subsample':0.9,
                    'n_estimators':round(best_iteration),
                    'seed': SEED,
                    'early_stopping_rounds':None, 
                }

train_data_final = lgb.Dataset(X, y)
LGBM = lgb.train(lgb_params, train_data)

In [ ]:
print(LGBM)

In [ ]:
# telling wich model to use
explainer = shap.TreeExplainer(LGBM)
# Calculating the Shap values of X features
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values[1], X, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X)

#Are you going to stick up for your teammates in this competition?

#This is what you call sticking up for your teammates

https://www.youtube.com/watch?v=i8wkptMptOE 

<iframe width="956" height="538" src="https://www.youtube.com/embed/i8wkptMptOE" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

#Since the Label Encoder above didn't work, I encoded again so that I could see the density of some features.

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in plays.columns:
    if plays[c].dtype=='float16' or  plays[c].dtype=='float32' or  plays[c].dtype=='float64':
        plays[c].fillna(plays[c].mean())

#fill in -999 for categoricals
plays = plays.fillna(-999)
# Label Encoding
for f in plays.columns:
    if plays[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(plays[f].values))
        plays[f] = lbl.transform(list(plays[f].values))
        
print('Labelling done.')

In [ ]:
#Code by Tudor Ioan  https://www.kaggle.com/tudoale1/heart-failure-prediction-using-adasys

def plotting(col): #function to plot data
    plt.figure(figsize=(5, 3))
    sns.distplot(plays[col])

for col in plays:
    plotting(col)

In [ ]:
brady = players[(players['displayName']=='Tom Brady')].reset_index(drop=True)
brady.head()

#I couldn't find Tom Brady. Anyhow....

![](https://media0.giphy.com/media/Nj4QsiCRlqIUuVji0B/200w.webp?cid=ecf05e47keq40ler7q04o5hamk99zdqniwcm3hhr49lzai89&rid=200w.webp&ct=g)